# Unsupervised Clustering
The following part of the notebook will contain the code used to investigate the potential value of a clustering approach for spam detection.

In [ ]:
%reset

In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
import nltk
from sklearn.cluster import DBSCAN
import matplotlib as mpl
import matplotlib.pyplot as plt
import random
import scipy.spatial.distance
from scipy.special import softmax
from scipy.spatial.distance import cdist
from sklearn import metrics
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm.notebook import tqdm

In [2]:
def setup_mpl():
    mpl.rcParams['font.family'] = 'Times New Roman'
    return None
setup_mpl()

## Implementation of K-means & K-means ++

The following two code cells contain the K-means implementation and the centroid initialization scheme, which is K-means ++. The specific approach of the initalization is described in the docstring of the plus-plus function.

In [8]:
def plus_plus(X, k, random_state=42, verbose=True):
    '''
    ##########################
    #k-means++ initialization#
    ##########################
    
    The approach:
         * 1st centroid is chosen uniformly at random from the observations.
         * Subsequently, the remaining centroids are chosen from the remaining observation with probability
           proportionally with the squared distance to the closest existing centroid
    
    Parameters:
        * X : Observations, X \in R^{observations x features}
        * k : Number of centroids
        * random_state : seed
    
    by Christian Djurhuus
    '''
    np.random.seed(random_state)
    
    # Allocating memory
    centroids = np.zeros((k, X.shape[1]))
    
    # Sampling first centroid uniformly at random from observations
    indicies = [i for i in range(X.shape[0])]
    first_idx = np.random.choice(indicies, size=1)
    indicies.remove(first_idx)
    centroids[0,:] = X[first_idx, :]

    # Determining remaining number of centroids:
    for i in range(1,k):
        if verbose:
            print(f'Number of centroids defined: {i+1}')
        # Compute distance between all observations and existing centroids
        pair_dist = (((np.expand_dims(X[indicies,:], 1)-centroids[:i,:]+1e-06)**2).sum(-1))
        #pair_dist = cdist(X[indicies,:], centroids[:i,:], metric='euclidean')

        # Probabilities:
        if pair_dist.ndim==1:
            #only one centroid available. Hence, dist to nearest centroid is just pair dist
            dist = pair_dist
            #probs = (np.exp(dist) / np.sum(np.exp(dist), axis=0))
            probs = dist/dist.sum()
        else:
            dist = pair_dist.min(axis=1)
            #probs = (np.exp(dist) / np.sum(np.exp(dist), axis=0))
            probs = dist/dist.sum()


        #Selecting one of the remaining observations
        selected_idx = np.random.choice(indicies,size=1, p=probs)
        indicies.remove(selected_idx)
        centroids[i, :] = X[selected_idx, :]
    
    return centroids

In [9]:
def kmeans(X, k, random_state=42, tot=1e-4, n_init=10, verbose=True):
    '''
    ##########################
    #k-means#
    ##########################
    
    Function run n_init number of k-means clustering and return the result with smallest inertia.
    The implementation have utilized numpy's broadcasting functionalities to increase 
    computational speed w.r.t computing pairwise distances.
    
    Parameters:
        * X : Observations, X \in R^{observations x features}
        * k : Number of centroids
        * random_state : seed
        * tot : tolerance criteria for loose convergence criteria
        * n_init : number of K-means run 
    
    by Christian Djurhuus
    '''
    #Substracting mean of data
    X -= X.mean(axis=0)

    best_inertia = 1e6

    #run n_init number of the kmeans algorithm and return clusters with minimum inertia
    for i in range(1, n_init+1):
        if verbose:
            print(f'Kmeans run no. {i}')
        
        #Determining centroids using kmeans++
        centroids = plus_plus(X=X, k=k, random_state=i*random_state, verbose=verbose)

        #initial placeholder
        prev_centroids = np.zeros(centroids.shape)
        assignments = np.zeros(X.shape[0])

        #Run until convergence
        itr = 1
        diff = 1e5
        while not np.allclose(prev_centroids, centroids):
            if verbose:
                print(f'Iteration number: {itr} - diff {diff}')
            prev_diff = diff
            prev_assignments = assignments

            #Using broadcasting to compute pairwise distances between observations and centroids
            dists=((np.expand_dims(X, 1)-centroids+1e-06)**2).sum(-1)**0.5
            assignments = dists.argmin(axis=1)
            prev_centroids = centroids.copy()

            #Update centroid position
            for idx in range(k):
                centroids[idx, :] = X[np.where(assignments==idx)].mean(axis=0)

            itr += 1


            #Distance between previous centroids and current
            diff = ((((prev_centroids - centroids + 1e-6)**2).sum(-1))**0.5).sum()

            #Early stopping when converged
            if np.array_equal(prev_assignments, assignments): #Check for strict convergence
                break

            if diff < tot: #Check for loose convergence
                break

        #Computing inertia
        #Sum of squared distance between each sample and its assigned center.
        inertia = 0
        for idx in range(k):

            inertia += np.sum((np.expand_dims(X[np.where(assignments==idx)],1) - centroids[idx, :]) ** 2, axis=0).sum() #squared dist

        if inertia < best_inertia:
            best_inertia = inertia
            if verbose:
                print(f'Current best inertia: {best_inertia}')
            best_assignments = assignments
            best_centroids = centroids

    return best_assignments, best_centroids, best_inertia

The following function defines the performance metrics utilized within this project:

In [10]:
def performance_metrics(X, assign, labels):
    ARI = metrics.adjusted_rand_score(labels, assign)
    NMI = metrics.adjusted_mutual_info_score(labels, assign)
    DBI = metrics.davies_bouldin_score(X, assign)
    
    print(f'ARI: {ARI:.3f}')
    print(f'NMI: {NMI:.3f}')
    print(f'DBI: {DBI:.3f}')
    print('\n')
    return ARI, NMI, DBI

## SMS

Reading the cleaned sms data:

In [12]:
df = pd.read_csv('../data/clean/clean_completeSpamAssassin.csv')
df['tokens'] = df['tokens'].apply(literal_eval)
df.head()

,text,label,tokens,str_tokens
0,\nSave up to 70% on Life Insurance.\nWhy Spend...,spam,"[save, life, insur, spend, life, quot, save, e...",save life insur spend life quot save ensur fam...
1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,spam,"[fight, risk, cancer, url, slim, guarante, los...",fight risk cancer url slim guarante lose lb da...
2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,spam,"[fight, risk, cancer, url, slim, guarante, los...",fight risk cancer url slim guarante lose lb da...
3,##############################################...,spam,"[adult, club, offer, free, membership, instant...",adult club offer free membership instant acces...
4,I thought you might like these:\n1) Slim Down ...,spam,"[thought, might, like, slim, guarante, lose, l...",thought might like slim guarante lose lb day u...


Vectorizing the documents using TFIDF. To be able to use our own tokenized data as input, an identity tokenizer is defined in given to the sklean TFIDF implementations. This prohibit sklearn from conducting any extra preprocessing, such that we only use our own preprocessing procedure.

In [13]:
#TDIDF
def identity_tokenizer(text):
    return text

vectorizer = TfidfVectorizer(tokenizer=identity_tokenizer,
                             lowercase=False
                            )

vecs = vectorizer.fit_transform(df['tokens'])
feature_names = vectorizer.get_feature_names_out()
dense = vecs.todense()
lst1 = dense.tolist()
TDM = pd.DataFrame(lst1, columns=feature_names).dropna()
X = vecs

### Clustering in Original Space

#### Hyperparameter tuning

First we will choose the most suitable $K$ in K-means by using the obtained inertia with varying values of $K$. The iniertia describes the sum of squared distance between each sample and its assigned centroid. Hence, a low inertia will indicate more well-defined clusters w.r.t inner distance:

In [ ]:
inertias = np.zeros(8)
best_inertia = 1e5
for i, k in tqdm(enumerate(np.arange(2,10,1))):
    assign, centroids, inertia = kmeans(X, k=k, random_state=2, tot=1e-4, verbose=False)
    inertias[i] = inertia
    if inertia < best_inertia:
        best_assign = assign
        best_centroids = centroids
        best_inertia = inertia

0it [00:00, ?it/s]

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(np.arange(2,10,1), inertias)
ax.set_xlabel('K')
ax.set_ylabel('Inertia')
plt.show()

Usually, we would choose the value of $K$ associated with the highest curvature. However, given that curve resembles a straight line and only marginal gains appear with increasing $K$, we decide to use $K=2$.

Subsequently, we will choose the most suitable $\epsilon$ value used in DBSCAN by visualising the distance between the n nearest neighbours as a function of n. $\epsilon$ is choosed where the curve express the highest curvature:

In [ ]:
from sklearn.neighbors import NearestNeighbors

n_neighbors = 10
nearest_neighbors = NearestNeighbors(n_neighbors=n_neighbors)
neighbors = nearest_neighbors.fit(X)
distances, indices = neighbors.kneighbors(X)
distances = np.sort(distances[:,n_neighbors-1], axis=0)
fig = plt.figure(figsize=(5, 5))
plt.plot(distances)
plt.xlabel("n")
plt.ylabel("Distance")
plt.show()

This approach appeared to be a bit more challenging than selecting $K$, we see that the highest curvature is with a very small epsilon value. Different epsilon values was also tried and the best appeared to be $\epsilon=0.1$, this is also true for all the other experiments. 

#### Performing Clustering

Conducting the clustering using the hyperparameters identified above:

In [ ]:
from sklearn.cluster import DBSCAN
kmeans_assign, centroids, _ = kmeans(X, k=2, random_state=2, tot=1e-4, verbose=False)
clustering_db = DBSCAN(eps=0.1, min_samples=2).fit(X)
dbscan_assign = clustering_db.labels_

Evaluating performance:

In [ ]:
labels = df.label.replace({'ham':0, 'spam':1}).values
performance_metrics(X.toarray(), kmeans_assign, labels)
performance_metrics(X.toarray(), dbscan_assign, labels)

### Latent Semantic Analysis


Computing the truncated SVD that approximately explains 50% of the variance.

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=300)
X_svd = svd.fit_transform(X)
var_explained = svd.explained_variance_ratio_.sum()
print(var_explained)

#### Hyperparameter tuning

Selecting $K$:

In [ ]:
inertias = np.zeros(8)
best_inertia = 1e5
for i, k in enumerate(np.arange(2,10,1)):
    assign, centroids, inertia = kmeans(X_svd, k=k, random_state=2, tot=1e-4, verbose=False)
    inertias[i] = inertia
    if inertia < best_inertia:
        best_assign = assign
        best_centroids = centroids
        best_inertia = inertia

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(np.arange(2,10,1), inertias)
ax.set_xlabel('K')
ax.set_ylabel('Inertia')
plt.show()

Selecting $\epsilon$:

In [ ]:
from sklearn.neighbors import NearestNeighbors
n_neighbors = 10
nearest_neighbors = NearestNeighbors(n_neighbors=n_neighbors)
neighbors = nearest_neighbors.fit(X_svd)
distances, indices = neighbors.kneighbors(X_svd)
distances = np.sort(distances[:,n_neighbors-1], axis=0)
fig = plt.figure(figsize=(5, 5))
plt.plot(distances)
plt.xlabel("n")
plt.ylabel("Distance")
plt.show()

#### Performing Clustering
Conducting clustering:

In [ ]:
clustering_db = DBSCAN(eps=0.1, min_samples=2).fit(X_svd)
dbscan_assign = clustering_db.labels_
kmeans_assign, centroids, _ = kmeans(X_svd, k=2, random_state=2, tot=1e-4, n_init=10, verbose=False)

Evaluating performance

In [ ]:
labels = df.label.replace({'ham':0, 'spam':1}).values
performance_metrics(X_svd, kmeans_assign, labels)
performance_metrics(X_svd, dbscan_assign, labels)

### Cosine Distance

In [ ]:
from sklearn.metrics import pairwise_distances
dist = pairwise_distances(X, metric='cosine')

#### Hyperparameter tuning
Selecting $K$:

In [ ]:
inertias = np.zeros(8)
best_inertia = 1e5
for i, k in tqdm(enumerate(np.arange(2,10,1))):
    assign, centroids, inertia = kmeans(dist, k=k, random_state=2, tot=1e-4, verbose=False)
    inertias[i] = inertia
    if inertia < best_inertia:
        best_assign = assign
        best_centroids = centroids
        best_inertia = inertia

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(np.arange(2,10,1), inertias)
ax.set_xlabel('K')
ax.set_ylabel('Inertia')
plt.show()

Selecting $\epsilon$:

In [ ]:
from sklearn.neighbors import NearestNeighbors
nearest_neighbors = NearestNeighbors(n_neighbors=100)
neighbors = nearest_neighbors.fit(dist)
distances, indices = neighbors.kneighbors(dist)
distances = np.sort(distances[:,10], axis=0)
fig = plt.figure(figsize=(5, 5))
plt.plot(distances)
plt.xlabel("n")
plt.ylabel("Distance")
#plt.xlim([0, 500])
plt.savefig("Distance_curve.png", dpi=300)

#### Performing Clustering
Conducting clustering:

In [ ]:
kmeans_assign, centroids_kmeans, _ = kmeans(dist, k=2, random_state=2, tot=0.0001, n_init=10, verbose=False)
dist = pairwise_distances(X, metric='cosine')
clustering_db = DBSCAN(eps=0.1, min_samples=2, metric='precomputed').fit(dist)
dbscan_assign = clustering_db.labels_

Evaluating performance:

In [ ]:
labels = df.label.replace({'ham':0, 'spam':1}).values
performance_metrics(dist, kmeans_assign, labels)
performance_metrics(dist, dbscan_assign, labels)

### Spectral Clustering 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(X)

In [ ]:
# Define affinity matrix
A = similarity

#Setting diagonal elements to 0.0 and removing documents with no similarity with other documents
np.fill_diagonal(A, 0.0)
row_sum = A.sum(0)
removed_idx = np.where(A.sum(0)==0)[0]
A_reduced = A[row_sum != 0, :]
A_reduced = A_reduced[:, row_sum!=0]

assert np.where(A_reduced.sum(0)==0)[0].size == 0
assert np.where(A_reduced.sum(1)==0)[0].size == 0

# Define D as a diagonal matrix where element (i,i) corresponds to the sum of ith row in A
D = np.diag(A_reduced.sum(axis=0))
D_pow = np.diag(np.power(np.diagonal(D), -0.5))
# Define L=D^{-1/2}AD^{-1/2}
L = D_pow@A_reduced@D_pow

# Find the k largest eigenvectors of L
eigenvals, eigenvectors = np.linalg.eigh(L)

k = 10
k_largest = np.argpartition(eigenvals, -k)[-k:]

#Create matrix X_spec = [x1,...,xk]
X_spec = eigenvectors[:, k_largest]

# Define Y as X row normalized
Y = X_spec/np.linalg.norm(X_spec, axis=0)

row_sums = np.power(X_spec, 2).sum(axis=1)
#row_sums = X_spec.sum(axis=1)
Y = (X_spec / np.power(row_sums[:, np.newaxis], 0.5))
#Y = X_spec / row_sums[:, np.newaxis]
Y = np.nan_to_num(Y)

#### Hyperparameter tuning
Choosing $K$:

In [ ]:
inertias = np.zeros(8)
best_inertia = 1e5
for i, k in enumerate(np.arange(2,10,1)):
    assign, centroids, inertia = kmeans(Y, k=k, random_state=2, tot=1e-4)
    inertias[i] = inertia
    if inertia < best_inertia:
        best_assign = assign
        best_centroids = centroids
        best_inertia = inertia

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(np.arange(2,10,1), inertias, c='darkblue')
ax.set_xlabel('K')
ax.set_ylabel('Inertia')
plt.show()

#### Conducting clustering

In [ ]:
kmeans_spec_assign, centroids, _ = kmeans(Y, k=2, random_state=2, tot=1e-4, verbose=False)

Evaluating performance:

In [ ]:
labels = df.drop(removed_idx).label.replace({'ham':0, 'spam':1}).values
performance_metrics(Y, kmeans_spec_assign, labels)

## Emails
This section will conduct the exact same procedure as the above. Hence, the number of comments will be limited and for further explainations please see the section above.

In [ ]:
df = pd.read_csv('data/clean/clean_completeSpamAssassin.csv')
df['tokens'] = df['tokens'].apply(literal_eval)
df.head()

### Clustering in Original Space
Choosing $K$:

In [ ]:
inertias = np.zeros(8)
best_inertia = 1e5
for i, k in tqdm(enumerate(np.arange(2,10,1))):
    assign, centroids, inertia = kmeans(X, k=k, random_state=2, tot=1e-4, verbose=False)
    inertias[i] = inertia
    if inertia < best_inertia:
        best_assign = assign
        best_centroids = centroids
        best_inertia = inertia

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(np.arange(2,10,1), inertias)
ax.set_xlabel('K')
ax.set_ylabel('Inertia')
plt.show()

Choosing $\epsilon$:

In [ ]:
from sklearn.neighbors import NearestNeighbors

n_neighbors = 10
nearest_neighbors = NearestNeighbors(n_neighbors=n_neighbors)
neighbors = nearest_neighbors.fit(X)
distances, indices = neighbors.kneighbors(X)
distances = np.sort(distances[:,n_neighbors-1], axis=0)
fig = plt.figure(figsize=(5, 5))
plt.plot(distances)
plt.xlabel("n")
plt.ylabel("Distance")
plt.show()

#### Performing Clustering
Conducting clustering

In [ ]:
kmeans_assign, centroids, _ = kmeans(X, k=2, random_state=2, tot=1e-4, verbose=False)
clustering_db = DBSCAN(eps=0.1, min_samples=2).fit(X)
dbscan_assign = clustering_db.labels_

Evaluating performance:

In [ ]:
labels = df.label.replace({'ham':0, 'spam':1}).values
performance_metrics(X.toarray(), kmeans_assign, labels)
performance_metrics(X.toarray(), dbscan_assign, labels)

### Latent Semantic Analysis

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=)
X_svd = svd.fit_transform(X)
var_explained = svd.explained_variance_ratio_.sum()
print(var_explained)

#### Hyperparameter tuning
Choosing $k$:

In [ ]:
inertias = np.zeros(8)
best_inertia = 1e5
for i, k in enumerate(np.arange(2,10,1)):
    assign, centroids, inertia = kmeans(X_svd, k=k, random_state=2, tot=1e-4)
    inertias[i] = inertia
    if inertia < best_inertia:
        best_assign = assign
        best_centroids = centroids
        best_inertia = inertia

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(np.arange(2,10,1), inertias)
ax.set_xlabel('K')
ax.set_ylabel('Inertia')
plt.show()

Choosing $\epsilon$:

In [ ]:
from sklearn.neighbors import NearestNeighbors
nearest_neighbors = NearestNeighbors(n_neighbors=100)
neighbors = nearest_neighbors.fit(X_svd)
distances, indices = neighbors.kneighbors(X_svd)
distances = np.sort(distances[:,10], axis=0)
fig = plt.figure(figsize=(5, 5))
plt.plot(distances)
plt.xlabel("n")
plt.ylabel("Distance")
plt.savefig("Distance_curve.png", dpi=300)

### Performing clustering
Conducting clustering:

In [ ]:
kmeans_assign, centroids, _ = kmeans(X_svd, k=4, random_state=2, tot=1e-4, verbose=False)

clustering_db = DBSCAN(eps=0.1, min_samples=2).fit(X_svd)
dbscan_assign = clustering_db.labels_

Evaluating performance:

In [ ]:
labels = df.label.replace({'ham':0, 'spam':1}).values
performance_metrics(X_svd, kmeans_assign, labels)
performance_metrics(X_svd, dbscan_assign, labels)

### Cosine Distance

In [ ]:
dist = pairwise_distances(X, metric='cosine')

#### Hyperparameter tuning
Choosing $K$:

In [ ]:
inertias = np.zeros(8)
best_inertia = 1e5
for i, k in enumerate(np.arange(2,10,1)):
    assign, centroids, inertia = kmeans(dist, k=k, random_state=2, tot=1e-4)
    inertias[i] = inertia
    if inertia < best_inertia:
        best_assign = assign
        best_centroids = centroids
        best_inertia = inertia

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(np.arange(2,10,1), inertias)
ax.set_xlabel('K')
ax.set_ylabel('Inertia')
plt.show()

Choosing $\epsilon$:

In [ ]:
from sklearn.neighbors import NearestNeighbors
nearest_neighbors = NearestNeighbors(n_neighbors=100)
neighbors = nearest_neighbors.fit(dist)
distances, indices = neighbors.kneighbors(dist)
distances = np.sort(distances[:,10], axis=0)
fig = plt.figure(figsize=(5, 5))
plt.plot(distances)
plt.xlabel("Points")
plt.ylabel("Distance")
#plt.xlim([0, 500])
plt.savefig("Distance_curve.png", dpi=300)

#### Performing Clustering

Conducting clustering:

In [ ]:
kmeans_assign, centroids_kmeans, _ = kmeans(dist, k=4, random_state=2, tot=0.0001, n_init=10)

In [ ]:
dist = pairwise_distances(X, metric='cosine')
clustering_db = DBSCAN(eps=0.1, min_samples=2, metric='precomputed').fit(dist)
dbscan_assign = clustering_db.labels_

Evaluating performance:

In [ ]:
labels = df.label.replace({'ham':0, 'spam':1}).values
performance_metrics(dist, kmeans_assign, labels)
performance_metrics(dist, dbscan_assign, labels)

### Spectral Clustering

In [ ]:
similarity = cosine_similarity(X)

In [ ]:
# Define affinity matrix
A = similarity

#Setting diagonal elements to 0.0 and removing documents with no similarity with other documents
np.fill_diagonal(A, 0.0)
row_sum = A.sum(0)
removed_idx = np.where(A.sum(0)==0)[0]
A_reduced = A[row_sum != 0, :]
A_reduced = A_reduced[:, row_sum!=0]

assert np.where(A_reduced.sum(0)==0)[0].size == 0
assert np.where(A_reduced.sum(1)==0)[0].size == 0

# Define D as a diagonal matrix where element (i,i) corresponds to the sum of ith row in A
D = np.diag(A_reduced.sum(axis=0))
D_pow = np.diag(np.power(np.diagonal(D), -0.5))
# Define L=D^{-1/2}AD^{-1/2}
L = D_pow@A_reduced@D_pow

# Find the k largest eigenvectors of L
eigenvals, eigenvectors = np.linalg.eigh(L)

k = 10
k_largest = np.argpartition(eigenvals, -k)[-k:]

#Create matrix X_spec = [x1,...,xk]
X_spec = eigenvectors[:, k_largest]

# Define Y as X row normalized
Y = X_spec/np.linalg.norm(X_spec, axis=0)

row_sums = np.power(X_spec, 2).sum(axis=1)
#row_sums = X_spec.sum(axis=1)
Y = (X_spec / np.power(row_sums[:, np.newaxis], 0.5))
#Y = X_spec / row_sums[:, np.newaxis]
Y = np.nan_to_num(Y)

#### Hyperparameter tuning
Choosing $K$:

In [ ]:
inertias = np.zeros(8)
best_inertia = 1e5
for i, k in enumerate(np.arange(2,10,1)):
    assign, centroids, inertia = kmeans(Y, k=k, random_state=2, tot=1e-4)
    inertias[i] = inertia
    if inertia < best_inertia:
        best_assign = assign
        best_centroids = centroids
        best_inertia = inertia

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(np.arange(2,10,1), inertias, c='darkblue')
ax.set_xlabel('K')
ax.set_ylabel('Inertia')
plt.show()

#### Performing Clustering
Conducting clustering:

In [ ]:
kmeans_spec_assign, centroids, _ = kmeans(Y, k=3, random_state=2, tot=1e-4, verbose=False)

Evaluating performance:

In [ ]:
labels = df.drop(removed_idx).label.replace({'ham':0, 'spam':1}).values
performance_metrics(Y, kmeans_spec_assign, labels)

## Visualizing Clustering Results
The following section will visualize the best obtained clustering results (Spectral clustering with K-means) wil be visualised and analysed in more depth.

### Reorganized Affinity Matrix

The following is the reorganized pairwise cosine similarity matrix both using the ground truth partition and the cluster allocations:

In [ ]:
new_order = labels.argsort()

_, group_len = np.unique(labels, return_counts=True)


D = A_reduced[:, new_order][new_order]

fig, (ax1) = plt.subplots(figsize=(10,10), dpi=100)
ax1.spy(D, precision=0.5, markersize=0.5)
ax1.set_xticks([])
ax1.set_yticks([])

y = labels
N = A_reduced.shape[0]
counts=np.unique(y,return_counts=True)[1]
counts_j=np.unique(y,return_counts=True)[1]
cum_i=np.cumsum(counts)
cum_j=np.cumsum(counts_j)

for i in range(cum_i.shape[0]):
    x1=np.array([cum_i[i],cum_i[i]])
    y1=np.array([[0,N]])
    line = plt.Line2D(y1, x1, lw=1.5, color='k', alpha=0.8)
    ax1.add_line(line)

for i in range(cum_i.shape[0]):
    x1=np.array([cum_j[i],cum_j[i]])
    y1=np.array([[0,N]])
    line = plt.Line2D(x1, y1, lw=1.5, color='k', alpha=0.8)
    ax1.add_line(line)

fig.savefig('sim_matrix_ground_truth.png', dpi=100)

fig, (ax2) = plt.subplots(figsize=(10,10), dpi=100)

freq = np.flip(np.argsort(np.bincount(kmeans_spec_assign))[-(np.unique(kmeans_spec_assign).size):])
new_order = np.hstack((np.where(kmeans_spec_assign==freq[0])[0], 
                       np.where(kmeans_spec_assign==freq[1])[0],
                       np.where(kmeans_spec_assign==freq[2])[0]))

D = A[:, new_order][new_order]

ax2.spy(D, precision=0.5, markersize=0.5)
ax2.set_xticks([])
ax2.set_yticks([])

y = kmeans_spec_assign
N = len(y)
counts=np.unique(y,return_counts=True)[1][freq]
counts_j=np.unique(y, return_counts=True)[1][freq]
cum_i=np.cumsum(counts)
cum_j=np.cumsum(counts_j)

for i in range(cum_i.shape[0]):
    x1=np.array([cum_i[i],cum_i[i]])
    y1=np.array([[0,N]])
    line = plt.Line2D(y1, x1, lw=1.5, color='k', alpha=0.8)
    ax2.add_line(line)

for i in range(cum_j.shape[0]):
    x1=np.array([cum_j[i],cum_j[i]])
    y1=np.array([[0,N]])
    line = plt.Line2D(x1, y1, lw=1.5, color='k', alpha=0.8)
    ax2.add_line(line)
    
plt.show()

### PCA

In [ ]:
from sklearn import decomposition

The following figures visualize the PCA of the Emails dataset annotated by the ground truth class and the cluster allocations:

In [ ]:
X_reduced = np.delete(X.toarray(), removed_idx, axis=0)
pca = decomposition.PCA(n_components=2)
pca.fit(X_reduced)
pca_emb = pca.transform(X_reduced)

In [ ]:
ham_idx = np.where(labels==0)[0]
spam_idx = np.where(labels==1)[0]


fig, ax1 = plt.subplots(figsize=(10,10), dpi=100)
clust1 = np.where(kmeans_spec_assign==0)[0]
clust2 = np.where(kmeans_spec_assign==1)[0]
clust3 = np.where(kmeans_spec_assign==2)[0]

ax1.scatter(pca_emb[:,0][ham_idx], 
            pca_emb[:,1][ham_idx],
            c='lightblue',
            s=5,
            label='ham')
ax1.scatter(pca_emb[:,0][spam_idx], 
            pca_emb[:,1][spam_idx],
            c='lightcoral',
            s=5,
            label='spam')

ax1.set_xlabel('PC1', fontsize=18)
ax1.set_ylabel('PC2', fontsize=18)
#ax1.set_title('Ground truth')

ax1.legend(prop={'size': 20})
fig.savefig('pca_embedding_ground_truth.png', dpi=100)

fig, ax2 = plt.subplots(figsize=(10,10), dpi=100)

ax2.scatter(pca_emb[:,0][clust2], 
            pca_emb[:,1][clust2],
            c='lightblue',
            s=5,
            label='cluster 1')
ax2.scatter(pca_emb[:,0][clust1], 
            pca_emb[:,1][clust1],
            c='lightcoral',
            s=5,
            label='cluster 2')
ax2.scatter(pca_emb[:,0][clust3], 
            pca_emb[:,1][clust3],
            c='khaki',
            s=5,
            label='cluster 3')

ax2.set_xlabel('PC1', fontsize=18)
ax2.set_ylabel('PC2', fontsize=18)
#ax2.set_title('Community assignment')
legend = ax2.legend(prop={'size': 20})
fig.savefig('pca_embedding_community.png', dpi=100)
plt.show()

## Missclassifications
By inspecting the visualisations of the clusters both in the PCA figures and the reordered pairwise similarity/affinity matrix, it seems as if the spectral clustering algorithmm have identified three clusters. One of which is mainly composed of ham mails, while the other two appear to be composed of spam mails. As a first step let us validate these indications:

In [ ]:
clust1_dist = np.unique(labels[np.where(kmeans_spec_assign==0)[0]], return_counts=True)
clust2_dist = np.unique(labels[np.where(kmeans_spec_assign==1)[0]], return_counts=True)
clust3_dist = np.unique(labels[np.where(kmeans_spec_assign==2)[0]], return_counts=True)

In [ ]:
for dist in [clust1_dist, clust2_dist, clust3_dist]:
    fig, ax = plt.subplots(figsize=(5,5), dpi=100)
    bar = ax.bar(['ham', 'spam'], dist[1], color=['lightblue', 'lightcoral'])
    
    # Add counts above the two bar graphs
    for rect in bar:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height/(np.sum(dist[1])):.3f}%', ha='center', va='bottom')

    plt.show()

The above illustrations verify the earlier indications.

As the next step in our analysis we will take a deeper look into where the classifications go wrong, and try to deduce why the misclassification might occur. First we will look into the non identified spam mails, that have been assigned to cluster with a majority of ham mails.

In [ ]:
from wordcloud import WordCloud
idx = np.where(kmeans_spec_assign==1)[0]
clust2 = labels[idx]

feature_names = vectorizer.get_feature_names_out()
TDM = pd.DataFrame(X_reduced, columns=feature_names).dropna()
miss_classified = TDM.iloc[idx[np.where(clust2==1)[0]], :]

clust_cloud = WordCloud(background_color="white", width=1600, height=800).generate_from_frequencies(miss_classified.T.sum(axis=1))

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), dpi=100)
ax.imshow(clust_cloud)
ax.axis("off")
ax.set_title('TFIDF weighted words \n Missclassified SPAM')
#plt.savefig('wordcloud_miss_spam.png',dpi=100)
plt.show()

It definitely stands out as SPAM. We will try to go through a few examples:

In [ ]:
miss_classified_text = df.drop(removed_idx).iloc[idx[np.where(clust2==1)[0]],:].text.values

In [ ]:
miss_classified_text[0]

In [ ]:
miss_classified_text[2]

In [ ]:
miss_classified_text[30]

A possible explanation of the algorithm's shortcoming can be our approach to handle URL links, given that the wordcloud of the TFIDF weighted words highlights the importance of words such as URL or link among the missclasified spam mails. Let us try to investigate the amount of URLs:

In [ ]:
import re
tot_urls = 0
for text in miss_classified_text:
    tot_urls += len(re.findall(r'(https?://[^\s]+)', text))

In [ ]:
print(f'The average number of URLs: {tot_urls/len(miss_classified_text):.2f}')

Let us compare this number to the spam mails of the two other clusters:

In [ ]:
clust1 = labels[np.where(kmeans_spec_assign == 0)[0]]
clust3 = labels[np.where(kmeans_spec_assign == 2)[0]]

for clust in [clust1, clust3]:
    temp_text = df.drop(removed_idx).iloc[idx[np.where(clust==1)[0]],:].text.values

    tot_urls = 0
    for text in temp_text:
        tot_urls += len(re.findall(r'(https?://[^\s]+)', text))
    print(f'The average number of URLs: {tot_urls/len(temp_text):.2f}')

Interestingly, one can see that there on average is more links/URLs in the missclassified spam than in the correctly identified spam. Hence, a possible explanation could in fact be that this shortcoming arises due to our initial manipulation of URLs.

A second thing we can look into is the ham mails being classified as spam:

In [ ]:
miss_classified_c1 = TDM.iloc[idx[np.where(clust1==0)[0]], :]
miss_classified_c2 = TDM.iloc[idx[np.where(clust3==0)[0]], :]

clust1_cloud = WordCloud(background_color="white", width=1600, height=800).generate_from_frequencies(miss_classified_c1.T.sum(axis=1))
clust2_cloud = WordCloud(background_color="white", width=1600, height=800).generate_from_frequencies(miss_classified_c2.T.sum(axis=1))

In [ ]:
fig, ax = plt.subplots(figsize=(10,10), dpi=100)
ax.imshow(clust1_cloud)
ax.axis("off")
plt.savefig('wordcloud_miss_hamm_c1.png', dpi=100)
#ax.set_title('TFIDF weighted words \n Missclassified HAM (cluster 1)')
plt.show()

fig, ax = plt.subplots(figsize=(10,10), dpi=100)
ax.imshow(clust2_cloud)
ax.axis("off")
#ax.set_title('TFIDF weighted words \n Missclassified HAM (cluster 3)')
#fig.tight_layout()
plt.savefig('wordcloud_miss_hamm_c2.png', dpi=100)
plt.show()

## Wordclouds for each cluster

In [ ]:
clust1_idx = np.where(kmeans_spec_assign==0)[0]
clust2_idx = np.where(kmeans_spec_assign==1)[0]
clust3_idx = np.where(kmeans_spec_assign==2)[0]

In [ ]:
for i, clust in enumerate([clust1_idx, clust2_idx, clust3_idx]):
    miss_classified = TDM.iloc[clust,:]
    clust_cloud = WordCloud(background_color="white", width=1600, height=800).generate_from_frequencies(miss_classified.T.sum(axis=1))
    
    fig, ax = plt.subplots(figsize=(10,10), dpi=100)
    ax.imshow(clust_cloud)
    ax.axis("off")
    #ax.set_title(f'TFIDF weighted words: cluster {i}')
    plt.savefig(f'wordcloud_cluster_{i+1}.png',dpi=100)
    plt.show()